In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [27]:
"""
from selenium import webdriver
import chromedriver_autoinstaller
import time

chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path

driver = webdriver.Chrome()
driver.get("http://www.python.org")

time.sleep(10)
driver.quit()
"""

'\nfrom selenium import webdriver\nimport chromedriver_autoinstaller\nimport time\n\nchromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists\n                                      # and if it doesn\'t exist, download it automatically,\n                                      # then add chromedriver to path\n\ndriver = webdriver.Chrome()\ndriver.get("http://www.python.org")\n\ntime.sleep(10)\ndriver.quit()\n'

In [80]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# Selenium için Chrome ayarları
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# WebDriver Manager ile otomatik ChromeDriver yönetimi
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Web sayfasını aç
url = "https://www.obilet.com/oteller/sehir-oteli-3-10-4/20250801-20250807/2ad?so=price-lowest"
driver.get(url)

print("Sayfa başarıyla yüklendi!")
driver.quit()


Sayfa başarıyla yüklendi!


In [81]:
%%time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

# Selenium için Chrome ayarları
chrome_options = Options()
chrome_options.add_argument("--headless")  # Tarayıcıyı gizli modda çalıştırır
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# WebDriver Manager ile otomatik ChromeDriver yönetimi
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Web sayfasını aç
url = "https://www.obilet.com/oteller/sehir-oteli-3-10-4/20250801-20250807/2ad?so=price-lowest"
driver.get(url)

# Sonsuz kaydırma için ayar
otel_bilgileri = []
previous_height = driver.execute_script("return document.body.scrollHeight")
max_oteller = 20  # Çekmek istediğiniz maksimum otel sayısı

while len(otel_bilgileri) < max_oteller:
    # Sayfanın sonuna kaydır
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # Yeni otellerin yüklenmesi için bekle

    # Sayfanın HTML içeriğini çek
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")

    # Şu ana kadar yüklenmiş tüm otelleri çek
    otel_listesi = soup.find_all("div", class_="hotel-item")
    
    # Her bir otelin bilgilerini çek
    for otel in otel_listesi:
        # Otel adı
        isim = otel.find("div", {"class": "hotel-item__name notranslate"})
        isim = isim.text.strip() if isim else "Bilinmiyor"

        # Otel adresi
        adres = otel.find("div", {"class": "hotel-location__address"})
        adres = adres.text.strip() if adres else "Bilinmiyor"

        # Şehir merkezine uzaklık
        merkeze_uzaklik = otel.find("div", {"class": "hotel-location__city-center-distance"})
        merkeze_uzaklik = merkeze_uzaklik.text.strip() if merkeze_uzaklik else "Bilinmiyor"

        # Yorum sayısı
        yorum_elementi = otel.find("span", {"class": "hotel-review__comment"})
        if yorum_elementi:
            yorum_sayisi = ''.join(filter(str.isdigit, yorum_elementi.text))  # Sadece rakamları al
        else:
            yorum_sayisi = "Bilinmiyor"

        # Puan
        puan = otel.find("span", {"class": "hotel-review__badge"})
        puan = puan.text.strip() if puan else "Bilinmiyor"

        # Yıldız sayısı
        otel_yildizi = otel.find("div", class_="hotel-item__star")
        if otel_yildizi:
            yildiz_sayisi = len(otel_yildizi.find_all("svg", class_="star"))
        else:
            yildiz_sayisi = "Bilinmiyor"

        # Fiyat
        fiyat = otel.find("div", {"class": "hotel-price__amount"})
        fiyat = fiyat.text.strip() if fiyat else "Bilinmiyor"

        # Ücretsiz İptal Bilgisi
        ucretsiz_iptal = otel.find("span")
        if ucretsiz_iptal and "Ücretsiz İptal" in ucretsiz_iptal.text:
            ucretsiz_iptal_bilgisi = ucretsiz_iptal.text.strip()
        else:
            ucretsiz_iptal_bilgisi = "Bilgi Yok"

        # Konaklama Tipi Bilgisi (Ultra Her Şey Dahil, vb.)
        konaklama_elementi = otel.find("div", {"class": "hotel-features__inner board-item"})
        if konaklama_elementi:
            konaklama_tipi_span = konaklama_elementi.find("span")
            konaklama_tipi = konaklama_tipi_span.text.strip() if konaklama_tipi_span else "Bilgi Yok"
        else:
            konaklama_tipi = "Bilgi Yok"

        # Otel bilgilerini bir sözlükte topla
        otel_bilgisi = {
            "İsim": isim,
            "Adres": adres,
            "Merkeze Uzaklık": merkeze_uzaklik,
            "Yorum Sayısı": yorum_sayisi,
            "Puan": puan,
            "Yıldız Sayısı": yildiz_sayisi,
            "Fiyat": fiyat,
            "Ücretsiz İptal": ucretsiz_iptal_bilgisi,
            "Konaklama Tipi": konaklama_tipi,
        }

        # Aynı otel bilgisini tekrar eklememek için kontrol et
        if otel_bilgisi not in otel_bilgileri:
            otel_bilgileri.append(otel_bilgisi)

    # Yeni yükleme olmadıysa döngüyü kır
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == previous_height:
        break  # Daha fazla otel yüklenmiyor
    previous_height = new_height

# Tarayıcıyı kapat
driver.quit()

# Verileri DataFrame'e çevir
df = pd.DataFrame(otel_bilgileri)

# DataFrame'i yazdır
print(df.head(2))  # İlk 2 kaydı göster


                     İsim                       Adres Merkeze Uzaklık  \
0     İstanbul Fair Hotel    Küçükçekmece, İstanbul -      Bilinmiyor   
1  Ethnica Hotel Old City  Sirkeci, Fatih, İstanbul -    Merkeze 1 km   

  Yorum Sayısı Puan  Yıldız Sayısı     Fiyat Ücretsiz İptal Konaklama Tipi  
0          156  8.3              3  5.850 TL      Bilgi Yok      Bilgi Yok  
1           15  9.4              3  6.636 TL      Bilgi Yok      Bilgi Yok  
CPU times: total: 641 ms
Wall time: 20 s


In [82]:
df

,İsim,Adres,Merkeze Uzaklık,Yorum Sayısı,Puan,Yıldız Sayısı,Fiyat,Ücretsiz İptal,Konaklama Tipi
0,İstanbul Fair Hotel,"Küçükçekmece, İstanbul -",Bilinmiyor,156,8.3,3,5.850 TL,Bilgi Yok,Bilgi Yok
1,Ethnica Hotel Old City,"Sirkeci, Fatih, İstanbul -",Merkeze 1 km,15,9.4,3,6.636 TL,Bilgi Yok,Bilgi Yok
2,Ayyıldızlar Otel,"Kadıköy, İstanbul -",Merkeze 4.4 km,117,5.3,3,6.772 TL,Bilgi Yok,Bilgi Yok
3,White Palace Hotel,"Bahçelievler, İstanbul -",Bilinmiyor,58,7.7,3,7.722 TL,Bilgi Yok,Bilgi Yok
4,Best Pasaport Pier Otel Kadıköy,"Moda, Kadıköy, İstanbul -",Merkeze 5 km,196,6.3,4,8.030 TL,Bilgi Yok,Bilgi Yok
5,Mardy Hotel,"Sefaköy, Küçükçekmece, İstanbul -",Bilinmiyor,80,7.7,3,8.466 TL,Bilgi Yok,Bilgi Yok
6,Mitani Suits,"Taksim, Beyoğlu, İstanbul -",Merkeze 0.3 km,48,7.8,0,8.559 TL,Bilgi Yok,Bilgi Yok
7,My Home Sultanahmet,"Sultanahmet, Fatih, İstanbul -",Merkeze 1 km,32,7.0,3,9.193 TL,Bilgi Yok,Kahvaltı Dahil
8,Taksim Palace Hotel Istanbul,"Taksim, Beyoğlu, İstanbul -",Merkeze 0.4 km,31,9.2,0,9.203 TL,Bilgi Yok,Bilgi Yok
9,Milano Hotel & Spa Sultanahmet,"Kumkapı, Fatih, İstanbul -",Merkeze 1.1 km,61,7.1,3,9.459 TL,Bilgi Yok,Kahvaltı Dahil


In [83]:
df_filtered = df[(df['Puan'] != 10)]
df_filtered

,İsim,Adres,Merkeze Uzaklık,Yorum Sayısı,Puan,Yıldız Sayısı,Fiyat,Ücretsiz İptal,Konaklama Tipi
0,İstanbul Fair Hotel,"Küçükçekmece, İstanbul -",Bilinmiyor,156,8.3,3,5.850 TL,Bilgi Yok,Bilgi Yok
1,Ethnica Hotel Old City,"Sirkeci, Fatih, İstanbul -",Merkeze 1 km,15,9.4,3,6.636 TL,Bilgi Yok,Bilgi Yok
2,Ayyıldızlar Otel,"Kadıköy, İstanbul -",Merkeze 4.4 km,117,5.3,3,6.772 TL,Bilgi Yok,Bilgi Yok
3,White Palace Hotel,"Bahçelievler, İstanbul -",Bilinmiyor,58,7.7,3,7.722 TL,Bilgi Yok,Bilgi Yok
4,Best Pasaport Pier Otel Kadıköy,"Moda, Kadıköy, İstanbul -",Merkeze 5 km,196,6.3,4,8.030 TL,Bilgi Yok,Bilgi Yok
5,Mardy Hotel,"Sefaköy, Küçükçekmece, İstanbul -",Bilinmiyor,80,7.7,3,8.466 TL,Bilgi Yok,Bilgi Yok
6,Mitani Suits,"Taksim, Beyoğlu, İstanbul -",Merkeze 0.3 km,48,7.8,0,8.559 TL,Bilgi Yok,Bilgi Yok
7,My Home Sultanahmet,"Sultanahmet, Fatih, İstanbul -",Merkeze 1 km,32,7.0,3,9.193 TL,Bilgi Yok,Kahvaltı Dahil
8,Taksim Palace Hotel Istanbul,"Taksim, Beyoğlu, İstanbul -",Merkeze 0.4 km,31,9.2,0,9.203 TL,Bilgi Yok,Bilgi Yok
9,Milano Hotel & Spa Sultanahmet,"Kumkapı, Fatih, İstanbul -",Merkeze 1.1 km,61,7.1,3,9.459 TL,Bilgi Yok,Kahvaltı Dahil


In [84]:
df_filtered = df_filtered[(df_filtered['Puan'] != 0)]
df_filtered

,İsim,Adres,Merkeze Uzaklık,Yorum Sayısı,Puan,Yıldız Sayısı,Fiyat,Ücretsiz İptal,Konaklama Tipi
0,İstanbul Fair Hotel,"Küçükçekmece, İstanbul -",Bilinmiyor,156,8.3,3,5.850 TL,Bilgi Yok,Bilgi Yok
1,Ethnica Hotel Old City,"Sirkeci, Fatih, İstanbul -",Merkeze 1 km,15,9.4,3,6.636 TL,Bilgi Yok,Bilgi Yok
2,Ayyıldızlar Otel,"Kadıköy, İstanbul -",Merkeze 4.4 km,117,5.3,3,6.772 TL,Bilgi Yok,Bilgi Yok
3,White Palace Hotel,"Bahçelievler, İstanbul -",Bilinmiyor,58,7.7,3,7.722 TL,Bilgi Yok,Bilgi Yok
4,Best Pasaport Pier Otel Kadıköy,"Moda, Kadıköy, İstanbul -",Merkeze 5 km,196,6.3,4,8.030 TL,Bilgi Yok,Bilgi Yok
5,Mardy Hotel,"Sefaköy, Küçükçekmece, İstanbul -",Bilinmiyor,80,7.7,3,8.466 TL,Bilgi Yok,Bilgi Yok
6,Mitani Suits,"Taksim, Beyoğlu, İstanbul -",Merkeze 0.3 km,48,7.8,0,8.559 TL,Bilgi Yok,Bilgi Yok
7,My Home Sultanahmet,"Sultanahmet, Fatih, İstanbul -",Merkeze 1 km,32,7.0,3,9.193 TL,Bilgi Yok,Kahvaltı Dahil
8,Taksim Palace Hotel Istanbul,"Taksim, Beyoğlu, İstanbul -",Merkeze 0.4 km,31,9.2,0,9.203 TL,Bilgi Yok,Bilgi Yok
9,Milano Hotel & Spa Sultanahmet,"Kumkapı, Fatih, İstanbul -",Merkeze 1.1 km,61,7.1,3,9.459 TL,Bilgi Yok,Kahvaltı Dahil


In [85]:
df_filtered = df_filtered [(df_filtered['Puan'] != 'Bilinmiyor')]
df_filtered

,İsim,Adres,Merkeze Uzaklık,Yorum Sayısı,Puan,Yıldız Sayısı,Fiyat,Ücretsiz İptal,Konaklama Tipi
0,İstanbul Fair Hotel,"Küçükçekmece, İstanbul -",Bilinmiyor,156,8.3,3,5.850 TL,Bilgi Yok,Bilgi Yok
1,Ethnica Hotel Old City,"Sirkeci, Fatih, İstanbul -",Merkeze 1 km,15,9.4,3,6.636 TL,Bilgi Yok,Bilgi Yok
2,Ayyıldızlar Otel,"Kadıköy, İstanbul -",Merkeze 4.4 km,117,5.3,3,6.772 TL,Bilgi Yok,Bilgi Yok
3,White Palace Hotel,"Bahçelievler, İstanbul -",Bilinmiyor,58,7.7,3,7.722 TL,Bilgi Yok,Bilgi Yok
4,Best Pasaport Pier Otel Kadıköy,"Moda, Kadıköy, İstanbul -",Merkeze 5 km,196,6.3,4,8.030 TL,Bilgi Yok,Bilgi Yok
5,Mardy Hotel,"Sefaköy, Küçükçekmece, İstanbul -",Bilinmiyor,80,7.7,3,8.466 TL,Bilgi Yok,Bilgi Yok
6,Mitani Suits,"Taksim, Beyoğlu, İstanbul -",Merkeze 0.3 km,48,7.8,0,8.559 TL,Bilgi Yok,Bilgi Yok
7,My Home Sultanahmet,"Sultanahmet, Fatih, İstanbul -",Merkeze 1 km,32,7.0,3,9.193 TL,Bilgi Yok,Kahvaltı Dahil
8,Taksim Palace Hotel Istanbul,"Taksim, Beyoğlu, İstanbul -",Merkeze 0.4 km,31,9.2,0,9.203 TL,Bilgi Yok,Bilgi Yok
9,Milano Hotel & Spa Sultanahmet,"Kumkapı, Fatih, İstanbul -",Merkeze 1.1 km,61,7.1,3,9.459 TL,Bilgi Yok,Kahvaltı Dahil


In [86]:
df_filtered = df_filtered[(df_filtered['Fiyat'] != 'Bilinmiyor')]
df_filtered

,İsim,Adres,Merkeze Uzaklık,Yorum Sayısı,Puan,Yıldız Sayısı,Fiyat,Ücretsiz İptal,Konaklama Tipi
0,İstanbul Fair Hotel,"Küçükçekmece, İstanbul -",Bilinmiyor,156,8.3,3,5.850 TL,Bilgi Yok,Bilgi Yok
1,Ethnica Hotel Old City,"Sirkeci, Fatih, İstanbul -",Merkeze 1 km,15,9.4,3,6.636 TL,Bilgi Yok,Bilgi Yok
2,Ayyıldızlar Otel,"Kadıköy, İstanbul -",Merkeze 4.4 km,117,5.3,3,6.772 TL,Bilgi Yok,Bilgi Yok
3,White Palace Hotel,"Bahçelievler, İstanbul -",Bilinmiyor,58,7.7,3,7.722 TL,Bilgi Yok,Bilgi Yok
4,Best Pasaport Pier Otel Kadıköy,"Moda, Kadıköy, İstanbul -",Merkeze 5 km,196,6.3,4,8.030 TL,Bilgi Yok,Bilgi Yok
5,Mardy Hotel,"Sefaköy, Küçükçekmece, İstanbul -",Bilinmiyor,80,7.7,3,8.466 TL,Bilgi Yok,Bilgi Yok
6,Mitani Suits,"Taksim, Beyoğlu, İstanbul -",Merkeze 0.3 km,48,7.8,0,8.559 TL,Bilgi Yok,Bilgi Yok
7,My Home Sultanahmet,"Sultanahmet, Fatih, İstanbul -",Merkeze 1 km,32,7.0,3,9.193 TL,Bilgi Yok,Kahvaltı Dahil
8,Taksim Palace Hotel Istanbul,"Taksim, Beyoğlu, İstanbul -",Merkeze 0.4 km,31,9.2,0,9.203 TL,Bilgi Yok,Bilgi Yok
9,Milano Hotel & Spa Sultanahmet,"Kumkapı, Fatih, İstanbul -",Merkeze 1.1 km,61,7.1,3,9.459 TL,Bilgi Yok,Kahvaltı Dahil


In [87]:
df_filtered['Puan']= pd.to_numeric(df_filtered['Puan'])

In [88]:
df_sorted_desc = df_filtered.sort_values(by='Puan')
df_sorted_desc

,İsim,Adres,Merkeze Uzaklık,Yorum Sayısı,Puan,Yıldız Sayısı,Fiyat,Ücretsiz İptal,Konaklama Tipi
2,Ayyıldızlar Otel,"Kadıköy, İstanbul -",Merkeze 4.4 km,117,5.3,3,6.772 TL,Bilgi Yok,Bilgi Yok
16,Monarch Hotel İstanbul,"Fatih, İstanbul -",Merkeze 1 km,39,5.7,3,10.071 TL,Bilgi Yok,Bilgi Yok
4,Best Pasaport Pier Otel Kadıköy,"Moda, Kadıköy, İstanbul -",Merkeze 5 km,196,6.3,4,8.030 TL,Bilgi Yok,Bilgi Yok
19,Green Garden İstanbul Hotel,"Bahçelievler, İstanbul -",Bilinmiyor,54,6.4,3,10.169 TL,Bilgi Yok,Bilgi Yok
20,Elit Suites,"Kurtköy, Pendik (İstanbul), İstanbul -",Bilinmiyor,211,6.4,5,10.200 TL,Bilgi Yok,Bilgi Yok
10,The Hera Premium Hotels,"Maltepe, İstanbul -",Bilinmiyor,160,6.5,4,9.600 TL,Bilgi Yok,Bilgi Yok
22,Valeria Deluxe Sultanahmet Hotel,"Beyazıt, Fatih, İstanbul -",Merkeze 0.9 km,28,6.6,3,10.238 TL,Bilgi Yok,Bilgi Yok
24,Magnificent Hotel,"Sultanahmet, Fatih, İstanbul -",Merkeze 1.3 km,35,6.8,3,10.248 TL,Bilgi Yok,Bilgi Yok
7,My Home Sultanahmet,"Sultanahmet, Fatih, İstanbul -",Merkeze 1 km,32,7.0,3,9.193 TL,Bilgi Yok,Kahvaltı Dahil
9,Milano Hotel & Spa Sultanahmet,"Kumkapı, Fatih, İstanbul -",Merkeze 1.1 km,61,7.1,3,9.459 TL,Bilgi Yok,Kahvaltı Dahil


In [89]:
df_reset = df_sorted_desc.reset_index()
df_reset

,index,İsim,Adres,Merkeze Uzaklık,Yorum Sayısı,Puan,Yıldız Sayısı,Fiyat,Ücretsiz İptal,Konaklama Tipi
0,2,Ayyıldızlar Otel,"Kadıköy, İstanbul -",Merkeze 4.4 km,117,5.3,3,6.772 TL,Bilgi Yok,Bilgi Yok
1,16,Monarch Hotel İstanbul,"Fatih, İstanbul -",Merkeze 1 km,39,5.7,3,10.071 TL,Bilgi Yok,Bilgi Yok
2,4,Best Pasaport Pier Otel Kadıköy,"Moda, Kadıköy, İstanbul -",Merkeze 5 km,196,6.3,4,8.030 TL,Bilgi Yok,Bilgi Yok
3,19,Green Garden İstanbul Hotel,"Bahçelievler, İstanbul -",Bilinmiyor,54,6.4,3,10.169 TL,Bilgi Yok,Bilgi Yok
4,20,Elit Suites,"Kurtköy, Pendik (İstanbul), İstanbul -",Bilinmiyor,211,6.4,5,10.200 TL,Bilgi Yok,Bilgi Yok
5,10,The Hera Premium Hotels,"Maltepe, İstanbul -",Bilinmiyor,160,6.5,4,9.600 TL,Bilgi Yok,Bilgi Yok
6,22,Valeria Deluxe Sultanahmet Hotel,"Beyazıt, Fatih, İstanbul -",Merkeze 0.9 km,28,6.6,3,10.238 TL,Bilgi Yok,Bilgi Yok
7,24,Magnificent Hotel,"Sultanahmet, Fatih, İstanbul -",Merkeze 1.3 km,35,6.8,3,10.248 TL,Bilgi Yok,Bilgi Yok
8,7,My Home Sultanahmet,"Sultanahmet, Fatih, İstanbul -",Merkeze 1 km,32,7.0,3,9.193 TL,Bilgi Yok,Kahvaltı Dahil
9,9,Milano Hotel & Spa Sultanahmet,"Kumkapı, Fatih, İstanbul -",Merkeze 1.1 km,61,7.1,3,9.459 TL,Bilgi Yok,Kahvaltı Dahil


In [90]:
df_reset = df_reset.drop(df.index[12:30])
df_reset

,index,İsim,Adres,Merkeze Uzaklık,Yorum Sayısı,Puan,Yıldız Sayısı,Fiyat,Ücretsiz İptal,Konaklama Tipi
0,2,Ayyıldızlar Otel,"Kadıköy, İstanbul -",Merkeze 4.4 km,117,5.3,3,6.772 TL,Bilgi Yok,Bilgi Yok
1,16,Monarch Hotel İstanbul,"Fatih, İstanbul -",Merkeze 1 km,39,5.7,3,10.071 TL,Bilgi Yok,Bilgi Yok
2,4,Best Pasaport Pier Otel Kadıköy,"Moda, Kadıköy, İstanbul -",Merkeze 5 km,196,6.3,4,8.030 TL,Bilgi Yok,Bilgi Yok
3,19,Green Garden İstanbul Hotel,"Bahçelievler, İstanbul -",Bilinmiyor,54,6.4,3,10.169 TL,Bilgi Yok,Bilgi Yok
4,20,Elit Suites,"Kurtköy, Pendik (İstanbul), İstanbul -",Bilinmiyor,211,6.4,5,10.200 TL,Bilgi Yok,Bilgi Yok
5,10,The Hera Premium Hotels,"Maltepe, İstanbul -",Bilinmiyor,160,6.5,4,9.600 TL,Bilgi Yok,Bilgi Yok
6,22,Valeria Deluxe Sultanahmet Hotel,"Beyazıt, Fatih, İstanbul -",Merkeze 0.9 km,28,6.6,3,10.238 TL,Bilgi Yok,Bilgi Yok
7,24,Magnificent Hotel,"Sultanahmet, Fatih, İstanbul -",Merkeze 1.3 km,35,6.8,3,10.248 TL,Bilgi Yok,Bilgi Yok
8,7,My Home Sultanahmet,"Sultanahmet, Fatih, İstanbul -",Merkeze 1 km,32,7.0,3,9.193 TL,Bilgi Yok,Kahvaltı Dahil
9,9,Milano Hotel & Spa Sultanahmet,"Kumkapı, Fatih, İstanbul -",Merkeze 1.1 km,61,7.1,3,9.459 TL,Bilgi Yok,Kahvaltı Dahil


In [91]:
df_reset.to_csv('istanbul.csv', index=False)